# Create Redshift

In [18]:
import boto3
import secrets
import string
import json
import time

In [2]:
rdsht = boto3.client('redshift')
scm = boto3.client('secretsmanager')

In [ ]:
alphabet = string.ascii_letters + string.digits
password = ''.join(secrets.choice(alphabet) for i in range(20)) # for a 20-character password

In [4]:
# redshift configuration
DBName='dev'
ClusterIdentifier='redsht-beyoung-20200917'
NodeType='ra3.4xlarge'
MasterUsername='awsuser'

# secrete manager configuration
scName=ClusterIdentifier
scString=json.dumps({'username':MasterUsername, 'password':password})

## Scretes manager for db password

In [ ]:
response = scm.create_secret(Name=scName, Description=scName, SecretString=scString)

In [6]:
response = scm.update_secret(SecretId=scName, SecretString=scString)

{'ARN': 'arn:aws:secretsmanager:us-west-2:232367251376:secret:redsht-beyoung-20200917-8ZgupW',
 'Name': 'redsht-beyoung-20200917',
 'VersionId': '1ff8a2f2-9389-4500-99b3-c86ae0a2b0dc',
 'ResponseMetadata': {'RequestId': '3e2b1bd7-8bb2-40ed-9141-c65acd83cc1d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 17 Sep 2020 03:50:53 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '177',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3e2b1bd7-8bb2-40ed-9141-c65acd83cc1d'},
  'RetryAttempts': 0}}

In [8]:
response = scm.get_secret_value(SecretId=scName)
db = json.loads(response['SecretString'])

awsuser
zcu6QsBPJSJuMEgEA3JB


In [10]:
response=rdsht.create_cluster(DBName=DBName, ClusterIdentifier=ClusterIdentifier, NodeType='ra3.4xlarge',
                     MasterUsername=db['username'],MasterUserPassword=db['password'], NumberOfNodes=2)
response

{'Cluster': {'ClusterIdentifier': 'redsht-beyoung-20200917',
  'NodeType': 'ra3.4xlarge',
  'ClusterStatus': 'creating',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'awsuser',
  'DBName': 'dev',
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-c81633b2',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-bcef6bda',
  'PreferredMaintenanceWindow': 'tue:06:30-tue:07:00',
  'PendingModifiedValues': {'MasterUserPassword': '****'},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 2,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouting': False,
  'IamRoles': [],
  'MaintenanceTrackName': 'current',
  'DeferredMaintenanceWindows': [],
  'NextMaintenanceWin

In [21]:
status = ''
while status != 'available':
    status=rdsht.describe_clusters(ClusterIdentifier=ClusterIdentifier)['Clusters'][0]['ClusterAvailabilityStatus']
    print(status)
    time.sleep(3)

available


In [23]:
# Connect to redshift
endpoint=rdsht.describe_clusters(ClusterIdentifier=ClusterIdentifier)['Clusters'][0]['Endpoint']
endpoint

{'Address': 'redsht-beyoung-20200917.cdc9adotlimv.us-west-2.redshift.amazonaws.com',
 'Port': 5439}

# Connect to redshift

In [24]:
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [27]:
rdsht_endpoint=endpoint['Address']
rdsht_port=endpoint['Port']
rdsht_user=db['username']
rdsht_pass=db['password']
engine_string=f"postgresql+psycopg2://{rdsht_user}:{rdsht_pass}@{rdsht_endpoint}:{rdsht_port}/{DBName}"
engine = create_engine(engine_string)

In [28]:
sql = """
select schemaname, tablename from pg_tables order by schemaname, tablename;
"""

In [29]:
tables = []
output = engine.execute(sql)
for row in output:
    tables.append(row)
tables

[('information_schema', 'sql_features'),
 ('information_schema', 'sql_implementation_info'),
 ('information_schema', 'sql_languages'),
 ('information_schema', 'sql_packages'),
 ('information_schema', 'sql_sizing'),
 ('information_schema', 'sql_sizing_profiles'),
 ('pg_catalog', 'padb_config_harvest'),
 ('pg_catalog', 'pg_aggregate'),
 ('pg_catalog', 'pg_am'),
 ('pg_catalog', 'pg_amop'),
 ('pg_catalog', 'pg_amproc'),
 ('pg_catalog', 'pg_attrdef'),
 ('pg_catalog', 'pg_attribute'),
 ('pg_catalog', 'pg_attribute_acl'),
 ('pg_catalog', 'pg_bar_ddllog'),
 ('pg_catalog', 'pg_bar_repos'),
 ('pg_catalog', 'pg_bar_state'),
 ('pg_catalog', 'pg_cast'),
 ('pg_catalog', 'pg_class'),
 ('pg_catalog', 'pg_class_extended'),
 ('pg_catalog', 'pg_conf'),
 ('pg_catalog', 'pg_constraint'),
 ('pg_catalog', 'pg_conversion'),
 ('pg_catalog', 'pg_database'),
 ('pg_catalog', 'pg_database_extended'),
 ('pg_catalog', 'pg_datashare'),
 ('pg_catalog', 'pg_datashare_consumers'),
 ('pg_catalog', 'pg_datashare_objects')

In [30]:
import pandas as pd
raw_data = pd.read_sql_query(text(sql), engine)

In [31]:
raw_data

,schemaname,tablename
0,information_schema,sql_features
1,information_schema,sql_implementation_info
2,information_schema,sql_languages
3,information_schema,sql_packages
4,information_schema,sql_sizing
...,...,...
807,pg_catalog,stv_xact
808,pg_catalog,systable_globaldict
809,pg_catalog,systable_schema
810,pg_catalog,systable_topology


# Clean up resources

In [34]:
# delete redshift cluster
respones=rdsht.delete_cluster(ClusterIdentifier=ClusterIdentifier, SkipFinalClusterSnapshot=True)
respones

{'Cluster': {'ClusterIdentifier': 'redsht-beyoung-20200917',
  'NodeType': 'ra3.4xlarge',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'awsuser',
  'DBName': 'dev',
  'Endpoint': {'Address': 'redsht-beyoung-20200917.cdc9adotlimv.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2020, 9, 17, 4, 2, 13, 656000, tzinfo=tzlocal()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-c81633b2',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-bcef6bda',
  'AvailabilityZone': 'us-west-2a',
  'PreferredMaintenanceWindow': 'tue:06:30-tue:07:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNode

In [33]:
# delete screte manager
respones=scm.delete_secret(SecretId=scName,ForceDeleteWithoutRecovery=True)
respones

{'ARN': 'arn:aws:secretsmanager:us-west-2:232367251376:secret:redsht-beyoung-20200917-8ZgupW',
 'Name': 'redsht-beyoung-20200917',
 'DeletionDate': datetime.datetime(2020, 9, 17, 6, 48, 22, 279000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'eff0ebf2-2054-46d9-8a4d-00257535cdee',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 17 Sep 2020 06:48:22 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '158',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'eff0ebf2-2054-46d9-8a4d-00257535cdee'},
  'RetryAttempts': 0}}